# This code will become obsolete since now finetuning will incorporate evaluation

In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset

In [21]:
# load model and processor
checkpoint = "openai_whisper-large/experiment_1/checkpoint-2000"
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")#checkpoint)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="hausa", task="transcribe")

# load streaming dataset and read first audio sample
#ds = load_dataset("mozilla-foundation/common_voice_9_0", "ha", split="test", streaming=True)
#ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
cache_dir="/data/users/kashrest/lrl-asr-experiments/data/fleurs"
data = load_dataset("google/fleurs", "ha_ng", split="test", cache_dir=cache_dir)

input_speech = next(iter(data))["audio"]

data_iter = iter(data)
transcriptions_val = []
for instance in data_iter:
    input_speech = instance["audio"]
    input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features

    # generate token ids
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    # decode token ids to text
    transcription = processor.batch_decode(predicted_ids)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    transcriptions_val.append(transcription)

Found cached dataset fleurs (/data/users/kashrest/asr-experiments/google___fleurs/ha_ng/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)
/data/users/kashrest/miniconda3/envs/asr/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [22]:
import json
with open("whisper-large-zeroshot_fleurs-test.jsonl", "w") as f:
    for transcript in transcriptions_val:
        json.dump(transcript, f)
        f.write("\n")

In [23]:
transcriptions = [elem[0] for elem in transcriptions_val]

In [24]:
cache_dir="/data/users/kashrest/lrl-asr-experiments/data/fleurs"
data = load_dataset("google/fleurs", "ha_ng", split="test", cache_dir=cache_dir)
labels = []
for elem in (iter(data)):
    labels.append(elem["transcription"])

Found cached dataset fleurs (/data/users/kashrest/asr-experiments/google___fleurs/ha_ng/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)


In [6]:
transcriptions[54]

'a lokacin 1920s hana ya mafirin jayi na yawancin dan ƙasa na ƙasashe shine na zaman lafiya da kebewa'

In [7]:
labels[54]

"a lokacin 1920s halaye mafi rinjaye na yawancin 'yan ƙasa da ƙasashe shine na zaman lafiya da keɓewa"

In [25]:
from datasets import load_metric

labels = []
for elem in iter(data):
    labels.append(elem["transcription"])

wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

wer_metric.compute(predictions=transcriptions, references=labels)

0.9641416405199462

In [26]:
cer_metric.compute(predictions=transcriptions, references=labels)

0.4114271804145222

In [ ]:
n = 102
print(f"Predicted transcript:\n{transcriptions[n]}\n\nOriginal:\n{labels[n]}")
print("\n****\n")